# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
city = "../output_data/city.csv"

# Read Purchasing File and store into Pandas data frame
city_df = pd.read_csv(city)
city_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Khanbari,27.3747,87.2039,50.50,74,100,1.57,NP,1612620907
1,1,Vaini,-21.2000,-175.2000,77.00,100,90,3.44,TO,1612620907
2,2,Jamestown,42.0970,-79.2353,16.00,67,90,21.85,US,1612620908
3,3,Tasiilaq,65.6145,-37.6368,32.00,93,90,21.85,GL,1612620909
4,4,Jeffersontown,38.1942,-85.5644,30.99,64,1,6.91,US,1612620909


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with api key
gmaps.configure(api_key=g_key)

# Set up Lat and Lng as locations and Humidity as Weight
humidity = city_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_df[["Lat", "Lng"]]

#Create Humidity Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Ideal Weather Conditions = temperature between 70 and 80 degrees, wind speed less than 10 mph
Ideal_city_df = city_df.loc[(city_df["Max Temp"] > 70) & (city_df["Max Temp"] < 80) & (city_df["Wind Speed"] < 10) & (city_df["Cloudiness"] == 0), :]
Ideal_city_df = Ideal_city_df.dropna(how="any")
Ideal_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Castro,-24.7911,-50.0119,72.88,57,0,1.03,BR,1612620909
45,45,Chui,-33.6971,-53.4616,73.94,59,0,9.71,UY,1612620916
59,59,Vao,-22.6667,167.4833,77.61,86,0,8.66,NC,1612620917
73,73,Jinji,23.2281,110.8261,71.37,43,0,1.10,CN,1612620919
75,75,Ribera,37.4984,13.2642,71.01,45,0,5.01,IT,1612620919


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create hotel list using Google Places and store first hotel in Data Frame
# Create hotel list
hotels = []

# Search for hotels related to each Ideal City using x to count through city list
for x in range(len(Ideal_city_df)):
    lat = Ideal_city_df.iloc[x][3]
    lng = Ideal_city_df.iloc[x][4]
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
    }
    
    google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(google_url, params=params).json()
    
# Use try and except to extract data
    try:
        hotels.append(response["results"][0]["name"])
    except:
        hotels.append("")
    #print(hotels)

# Print out dataframe with added column for the first Hotel
Ideal_city_df["Hotel Name"] = hotels
Ideal_city_df = Ideal_city_df.dropna(how="any")
Ideal_city_df.head(20)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,Castro,-24.7911,-50.0119,72.88,57,0,1.03,BR,1612620909,
45,45,Chui,-33.6971,-53.4616,73.94,59,0,9.71,UY,1612620916,
59,59,Vao,-22.6667,167.4833,77.61,86,0,8.66,NC,1612620917,
73,73,Jinji,23.2281,110.8261,71.37,43,0,1.10,CN,1612620919,
75,75,Ribera,37.4984,13.2642,71.01,45,0,5.01,IT,1612620919,
282,282,Lima,-12.0432,-77.0282,72.00,83,0,9.22,PE,1612620945,
297,297,Hermanus,-34.4187,19.2345,78.01,67,0,7.00,ZA,1612620748,Pardi Buzurg
344,344,Hadgāon,19.5000,77.6667,73.69,28,0,6.35,IN,1612620952,
420,420,Bosaso,11.2842,49.1816,76.80,64,0,6.40,SO,1612620962,
425,425,Aci Castello,37.5583,15.1346,75.20,68,0,8.05,IT,1612620963,Ingalagi


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Ideal_city_df.iterrows()]
locations = Ideal_city_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))